## LLM 調用

In [4]:
import os

from langchain.embeddings.base import Embeddings
from langchain.messages import (AIMessageChunk, HumanMessage, SystemMessage,
                                ToolMessage)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama import OllamaEmbeddings
from langchain_openai import ChatOpenAI
from langchain_postgres.vectorstores import PGVector
from openai import OpenAI
from pydantic import BaseModel, Field

from src.config.constant import (EMBEDDING_MODEL, OLLAMA_LOCAL, OLLAMA_URL,
                                 PG_COLLECTION, PROJECT_ROOT, SYSTEM_PROMPT)
from src.database import postgreSQL_conn as pgc

In [5]:
# 建立embedding連線
embeddings = OllamaEmbeddings(
    model=EMBEDDING_MODEL,
    base_url=OLLAMA_URL
)

# 載入向量資料庫
pg_url = pgc.connect_to_pgSQL()
vector_store = PGVector(
        embeddings=embeddings,
        collection_name=PG_COLLECTION,
        connection=pg_url,
        use_jsonb=True,
    )

In [ ]:
# 建立embedding類別
class LmStudioEmbeddings(Embeddings):
    def __init__(self, model_name, url):
        self.model_name = model_name
        self.url = url
        self.client = OpenAI(base_url=url, api_key="lm-studio")

    def embed_query(self, text: str):
        response = self.client.embeddings.create(input=text,model=self.model_name)
        return response.data[0].embedding

    def embed_documents(self, texts: list[str]):
        # 回傳多個文件的 embedding
        response = self.client.embeddings.create(input=texts,model=self.model_name)
        return [x.embedding for x in response.data]
        # return [self.model.encode(t).tolist() for t in texts]


class stream_chat_bot:
    def __init__(self, llm, tools):
        # 初始化對話機器人，傳入 LLM 與可用工具列表
        self.tools = tools
        # 將 LLM 綁定（bind）工具，使其具備自動呼叫工具的能力
        self.llm_with_tools = llm.bind_tools(tools)

        # 系統提示詞（System Prompt），用來設定 LLM 的角色與行為
        system_prompt = SYSTEM_PROMPT
        # 初始化訊息列表，第一條訊息是系統指令
        self.message = [SystemMessage(system_prompt)]

        # 將 LLM 的回應解析為純文字格式的工具
        self.str_parser = StrOutputParser()


    def chat_generator(self, text, display_data=False):
        """
        主對話生成函式（生成器形式）。
        逐步執行 LLM 回應與工具調用，並即時回傳每一步的結果。
        """
        # 將使用者的輸入加入訊息列表
        self.message.append(HumanMessage(text))

        while True:
            # 呼叫 LLM，傳入完整訊息歷史
            final_ai_message = AIMessageChunk(content="")
            for chunk in self.llm_with_tools.stream(self.message):
                final_ai_message += chunk
                if hasattr(chunk, 'content') and chunk.content:
                    yield self.str_parser.invoke(chunk)

            response = final_ai_message

            # 將 LLM 回應加入訊息列表
            self.message.append(response)

            # 檢查 LLM 是否要求呼叫工具
            is_tools_call = False
            for tool_call in response.tool_calls:
                is_tools_call = True

                if display_data:
                    # # 顯示 LLM 要執行的工具名稱與參數
                    msg = f'[執行]: {tool_call["name"]}({tool_call["args"]})\n-----------\n' #完整訊息
                    # # msg = f'[執行]: {tool_call["name"]}()\n\n' #簡易訊息
                    yield msg  # 使用 yield 讓結果能即時顯示在輸出中

                # 實際執行工具（根據工具名稱動態呼叫對應物件）
                tool_result = globals()[tool_call['name']].invoke(tool_call['args'])

                if display_data:
                    # # 顯示工具執行結果
                    msg = f'[結果]: {tool_result}\n-----------\n'
                    yield msg

                # 將工具執行結果封裝成 ToolMessage 回傳給 LLM
                tool_message = ToolMessage(
                    content=str(tool_result),          # 工具執行的文字結果
                    name=tool_call["name"],            # 工具名稱
                    tool_call_id=tool_call["id"],      # 工具呼叫 ID（讓 LLM 知道對應哪個呼叫）
                )
                # 將工具回傳結果加入訊息列表，提供 LLM 下一輪參考
                self.message.append(tool_message)

            # 若這一輪沒有任何工具呼叫，表示 LLM 已經生成最終回覆
            if is_tools_call == False:
                # 將 LLM 回應解析成純文字並輸出
                # yield self.str_parser.invoke(response)
                return  # 結束對話流程


    def chat(self, text, print_output=False):
        """
        封裝版對話函式。
        會收集 chat_generator 的所有輸出，並組合成完整的回覆字串。
        """
        msg = ''
        # 逐步取得 chat_generator 的產出內容
        for chunk in self.chat_generator(text):
            msg += f"{chunk}"
            if print_output:
                print(chunk, end='')
        # 回傳最終組合的對話內容
        return msg

In [ ]:
class GetQAInput(BaseModel):
    question: str = Field(description="查詢的問題文字")
    k: int = Field(default=2, description="要回傳的文件數量")

# # 舊版父子文件RAG
# @tool(args_schema=GetQAInput)
# def get_qa(question, n=10, k=2):
#     """
#     根據使用者提出的問題，從向量資料庫中檢索出最相關的 K 筆問答文件。

#     Args:
#         question (str): 查詢的問題文字。
#         n (int): 搜尋子文件的數量。
#         k (int): 要回傳的文件數量，預設為 2。

#     Returns:
#         documents: 檢索到的相似文件列表。
#     """
#     docs = vector_store.similarity_search(question, k=n)
#     seen_ids = set()
#     documents = []
#     for doc in docs:
#         if doc.metadata["parent_id"] not in seen_ids:
#             seen_ids.add(doc.metadata["parent_id"])
#             parent_docs = vector_store.similarity_search(query="",k=1,filter={"doc_id": doc.metadata["parent_id"]})
#             if len(parent_docs) > 0:
#                 documents.append(parent_docs[0])

#     return documents[:k]


@tool(args_schema=GetQAInput)
def get_qa(question, n=10, k=2):
    """
    根據使用者提出的問題，從向量資料庫中檢索出最相關的 K 筆問答文件。

    Args:
        question (str): 查詢的問題文字。
        n (int): 搜尋子文件的數量。
        k (int): 要回傳的文件數量，預設為 2。

    Returns:
        documents: 檢索到的相似文件列表。
    """
    # 檢索子文件
    child_docs = vector_store.similarity_search(question, k=n)

    # 提取父文件id
    unique_parent_ids = list(dict.fromkeys([
        doc.metadata["parent_id"] for doc in child_docs if "parent_id" in doc.metadata
    ]))

    target_ids = unique_parent_ids[:k]
    if not target_ids:
        return []

    # 批次查詢父文件
    parent_documents = vector_store.similarity_search(
        query="", 
        k=len(target_ids), 
        filter={"doc_id": {"$in": target_ids}} # 假設支援 $in 運算子
    )

    return parent_documents

In [ ]:
"""選擇主要LLM"""

model_select = input("請選擇使用模型:(1.地端gemma3-12b / 2.雲端gemini-3-flash)")
if str(model_select) == "1":
    model_name = 'gemma-3-12b-it'  # 指定模型名稱，模型名稱會根據下載的模型不同而改變

    base_url = 'http://192.168.0.109:1234/v1'  # LM Studio 本地伺服器的URL

    llm = ChatOpenAI(
        model=model_name,
        openai_api_key="not-needed",
        openai_api_base=base_url
    )

elif str(model_select) == "2":
    API_KEY = os.getenv("GOOGLE_API")
    model_name = 'gemini-3-flash-preview'

    llm = ChatGoogleGenerativeAI(
        model=model_name,
        google_api_key=API_KEY
    )

else:
    print("請輸入1或2選擇使用模型")

In [ ]:
# # 串接Gemini
# API_KEY = os.getenv("GOOGLE_API")
# model_name = 'gemini-3-flash-preview'

# llm = ChatGoogleGenerativeAI(
#     model=model_name,
#     google_api_key=API_KEY
# )

In [ ]:
# # 串接LM Studio
# model_name = 'gemma-3-12b-it'  # 指定模型名稱，模型名稱會根據下載的模型不同而改變

# base_url = 'http://192.168.0.109:1234/v1'  # LM Studio 本地伺服器的URL

# llm = ChatOpenAI(
#     model=model_name,
#     openai_api_key="not-needed",
#     openai_api_base=base_url
# )

In [9]:
bot = stream_chat_bot(llm, [get_qa])

In [10]:
for x in bot.chat_generator("上古卷軸online的評價如何", display_data=True):
    print(x, end='')

[執行]: get_qa({'question': '上古卷軸online的評價如何', 'k': 5})
-----------
[結果]: [Document(id='306130_p01835', metadata={'name': 'The Elder Scrolls Online', 'tags': 'RPG, MMORPG, Open World, Fantasy, Adventure, Multiplayer, Exploration, Singleplayer, Massively Multiplayer, Action, Character Customization, PvP, PvE, Lore-Rich, Story Rich', 'type': 'game', 'doc_id': '306130_p01835', 'genres': 'Action, Adventure, Massively Multiplayer, RPG', 'is_free': False, 'is_parent': True, 'languages': 'English, French, German, Russian, Spanish - Spain, Simplified Chinese', 'parent_id': '306130_p01835', 'platforms': 'windows, mac', 'categories': 'Multi-player, MMO, PvP, Online PvP, Co-op, Online Co-op, Steam Trading Cards, Captions available, In-App Purchases, Camera Comfort, Custom Volume Controls, Playable without Timed Input, Stereo Sound, Surround Sound, Partial Controller Support, HDR available, Family Sharing', 'developers': 'ZeniMax Online Studios', 'publishers': 'Bethesda Softworks', 'steam_appid': 30

In [8]:
for x in bot.chat_generator("請介紹CS:GO的遊戲內容"):
    print(x, end='')

作為 Steam 上的老玩家，我很樂意為您介紹這款經典中的經典。

《Counter-Strike: Global Offensive》（簡稱 **CS:GO**）是 Valve 開發的一款第一人稱射擊遊戲 (FPS)，也是全球電競史上最具影響力的作品之一。不過，有一點需要特別提醒您：**目前 CS:GO 已經正式升級並更名為《Counter-Strike 2 (CS2)》**。

以下是針對其遊戲核心內容的詳細介紹：

### 1. 遊戲核心機制：警匪對決
遊戲主要圍繞著兩大陣營的對抗：
*   **反恐小組 (Counter-Terrorists, CT)：** 目標是拆除炸彈、營救人質，或消滅所有恐怖分子。
*   **恐怖分子 (Terrorists, T)：** 目標是安置炸彈（C4）並保護其引爆，或守住人質不被救走。

### 2. 經典戰術玩法
*   **經濟系統：** 這是 CS 系列最獨特的設計。每一局開始時，玩家需要利用上一局賺取的金錢來購買武器（如 AK-47, AWP）、護甲及投擲物（閃光彈、煙霧彈、手榴彈）。如何管理團隊經濟（例如決定「存錢局」或「全買局」）是獲勝的關鍵。
*   **精準射擊與走位：** 遊戲極度要求反應速度與後座力控制（壓槍）。同時，聽音辨位、預瞄點位以及團隊間的報位資訊交換也非常重要。

### 3. 目前的現況：進化至 Counter-Strike 2 (CS2)
原本的 CS:GO 現在已免費升級為 **Counter-Strike 2**，帶來了多項重大技術革新：
*   **Source 2 引擎：** 畫面細節大幅提升，光影更加真實。
*   **動態煙霧彈：** 煙霧現在是三維物體，會與環境互動（例如子彈或手榴彈可以短暫炸開煙霧空洞），這徹底改變了戰術深度。
*   **全新的 CS 評級：** 更新了優先權模式（Premier mode）與全球排行榜，讓競爭更具透明度。
*   **物品繼承：** 玩家在 CS:GO 中擁有的所有外觀皮膚（Skins）都已完整移至 CS2，並在更強大的引擎下呈現更精美的視覺效果。

### 4. 遊戲特色總結
*   **類型：** 競技型 FPS、團隊戰略。
*   **付費模式：** **免費遊玩**（可加購「優先帳戶狀態」以獲得競技排名與物品掉落）。
*   **